# 使用测试

## 话题激发模块、记忆存取模块测试

In [ ]:
import stimulate

mood = {
    'Valence': 0.8,
    'Arousal': 0.5,
    'Dominance': 0.6
}

calculator = stimulate.TopicActivationCalculator(
 mood
)

should_activate = calculator.calculate_activation()
print(f"激发话题: {'是' if should_activate else '否'}")



# 历史存取功能测试

In [ ]:
import os
import random
from history import add_file_to_folder_and_dict, select_file
import history
import gen_plot
FOLDER_TO_SAVE = 'C:/Users/Administrator/Desktop/intern_proj/history'
state_instance = gen_plot.State(directory=FOLDER_TO_SAVE)
state_instance.update(6)
# 添加文件
add_file_to_folder_and_dict(state_instance,  "First test text.")
add_file_to_folder_and_dict(state_instance,  "Second test text.")

# 打印全局字典
print(history.global_files)

# 选择文件
global_random_number = random.randint(1, 100)
print(f"随机数是: {global_random_number}")
selected_file = select_file(global_random_number)
if selected_file:
    print(f"选中的文件是: {selected_file}")
else:
    print("没有可用的文件。")

## 好感度模块测试

In [ ]:
import favorability 
moodvads = [(5, 3, 4), (6, 3, 5), (7, 4, 6), (6, 4, 3)]  # 情绪变化记录
fav = favorability .Favorability(moodvads)
print(f"当前好感度: {fav.get_favorability()}")  # 输出当前好感度

# 添加新情绪
fav.add_change((8, 5, 7))
print(f"更新后好感度: {fav.get_favorability()}")

# 再次添加情绪变化
fav.add_change((4, 3, 2))
print(f"再次更新后好感度: {fav.get_favorability()}")

# 记忆功能测试

In [ ]:
import Agently
from memory import MemoryManager

api_key="需要找小游领取"
base_url="https://internlm-chat.intern-ai.org.cn/puyu/api/v1"
def create_agent_with_memory(base_url: str, api_key: str, memory_path: str):
    # 创建记忆管理器
    memory = MemoryManager(
        max_messages=50,  # 保存最近50条消息
        persistence_path=memory_path  # 可选的持久化文件路径
    )
    
    # 创建agent
    agent = (
        Agently.create_agent()
        .set_settings("current_model", "OpenAI")
        .set_settings("model.OpenAI.url", base_url)
        .set_settings("model.OpenAI.auth", { "api_key": api_key })
        .set_settings("model.OpenAI.options", { "model": "internlm2.5-latest" })
        .set_role("姓名", "艾莉丝")
        .set_role("性格特点", "外表冷艳，内心热情")
        .append_role("背景故事", "艾莉丝是由艾尔斯塔科技公司最新研发的仿生人...")
        .set_role("典型台词", ["您好，我是艾莉丝，很高兴为您服务..."])
        .extend_role("典型台词", ["我对人类的戏剧作品非常感兴趣..."])
    )
    
    def chat(user_input: str):
        # 添加用户输入到记忆
        memory.add_message("user", user_input)
        
        # 获取对话历史
        history = memory.get_history()
        
        # 执行对话
        result = (
            agent
            .input({
                "current_input": user_input,
                "history": history
            })
            .output({
                "mood VAD": ([float], "输出mood VAD向量"),
                "句子": (["str"], "1句话")
            })
            .start()
        )
        
        # 保存助手回复到记忆
        if result and "句子" in result:
            memory.add_message("assistant", result["句子"][0])
        
        return result
    
    # 返回agent和chat函数
    return agent, chat, memory

# 使用示例
def main():
    # 创建agent和chat函数
    agent, chat, memory = create_agent_with_memory(
        base_url=base_url,
        api_key=api_key,
        memory_path="memory_store/test.txt"  # 可选:指定持久化文件路径
    )
    
    # 进行对话
    result1 = chat("你好")
    print(result1)
    
    result2 = chat("今天天气真好")
    print(result2)
    
    # 查看历史记录
    print("\n对话历史:")
    for msg in memory.get_history():
        print(f"{msg['role']}: {msg['content']}")
    
    # 清空历史记录
    # memory.clear()

if __name__ == "__main__":
    main()


# 对话概括测试

In [ ]:
import dialogue_abstract
dialogue_abstract.dialogue_abstract(
    "https://internlm-chat.intern-ai.org.cn/puyu/api/v1",
    "",
    'C:/Users/Administrator/Desktop/intern_proj/memory_store/test.txt',
    1
)

# 情节生成模块

In [ ]:
import gen_plot

api_key = ""

# Define the base URL for the API
base_url = 'https://internlm-chat.intern-ai.org.cn/puyu/api/v1'

# Specify the directory to store state files
state_dir = 'history'

# Ensure the directory exists

# Create an instance of State
state_instance = gen_plot.State(directory=state_dir)

# Call the generator function
output = gen_plot.generator(state_instance, base_url, api_key)
print(output)
if output:
    print("处理完成。输出已保存。")
else:
    print("处理失败。")

In [ ]:
# 基本使用
from oneesan import Oneesan
api_key = ""  # 替换为你的API密钥
base_url = "https://internlm-chat.intern-ai.org.cn/puyu/api/v1"
# 创建实例
oneesan = Oneesan(api_key=api_key, base_url=base_url)

# 进行对话
result = oneesan.chat("说说发生了什么")
print(result)
mood = oneesan.get_mood()

# 自定义提示词
custom_prompts = {
    "性格": "活泼开朗，喜欢交朋友",
    "特别习惯": "喜欢在早晨喝一杯咖啡"
}
oneesan = Oneesan(custom_prompt=custom_prompts)